In [1]:
import pandas as pd
from textblob import TextBlob
from textblob import Word
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from matplotlib import pyplot as plt
import statsmodels.formula.api as smf
import geopandas as gpd
import urllib
import json
from dateutil.parser import parse
from datetime import datetime
import re
from collections import Counter
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn import metrics
import numpy as np


from nltk import word_tokenize
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('spanish')
from string import punctuation  
from nltk.corpus import stopwords  
from nltk import word_tokenize  
from nltk.data import load  
from string import punctuation  
from sklearn.feature_extraction.text import CountVectorizer       


pd.options.display.max_columns = 120
pd.options.display.max_rows = 120
%matplotlib inline

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/geoffperrin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/geoffperrin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Source: https://www.dataquest.io/blog/naive-bayes-tutorial/ & http://blog.manugarri.com/sentiment-analysis-in-spanish/

In [2]:
#stopword list to use
spanish_stopwords = stopwords.words('spanish')
#punctuation to remove
non_words = list(punctuation)  
#we add spanish punctuation
non_words.extend(['¿', '¡'])  
non_words.extend(map(str,range(10)))

In [3]:
data = '/Users/geoffperrin/Desktop/bloomberg_work/bogota_project/data/'

In [4]:
df_w_labels = pd.read_csv(data + 'quejas_English_w_labels.csv', encoding='utf-8')

In [5]:
df_w_labels = df_w_labels[df_w_labels.complaint_label!='?']
df_w_labels['complaint_binary'] = [1 if i == 'complaint' else 0 for i in df_w_labels['complaint_label']]
df_w_labels.head()

,Unnamed: 0,Numero petición,Sector,Entidad,Entidad Nueva,Trasladado?,Entidad definitiva,Dependencia,Tema,Subtema,Localidad de los hechos,UPZ de los hechos,Barrio de los hecho,Estrato de los hechos,Punto atención,Canal,Tipo petición,Asunto,Fecha ingreso,estado,inicio terminos,cierre,dias gestión,Fecha registro,Nombre peticionario,Condición del peticionario,Localidad del ciudadano,UPZ del ciudadano,Barrio del ciudadano,Estrato del ciudadano,Tipo de Peticionario,Opciones de Petición,Entidad que recibe,Entidad que traslada,Asunto_EN,complaint_label,complaint_binary
0,0,2112017,HABITAT,ACUEDUCTO - EAB,NaN,NaN,ACUEDUCTO - EAB,DIRECCION DE SERVICIO ADMINISTRATIVOS,SERVICIOS PUBLICOS,ATENCION Y SERVICIO A LA CIUDADANIA,4 - SAN CRISTOBAL,32 - SAN BLAS,BELLAVISTA SUR,2.0,NaN,WEB,DERECHO DE PETICIÓN DE INTERÉS PARTICULAR,BUENAS TARDES ES PARA SOLICITAR LA REPARACION ...,2017-01-02 00:00:00.000,SOLUCIONADO - POR RESPUESTA DEFINITIVA,2017-01-03 00:00:00.000,2017-01-23 12:59:50.415,14,2017-01-03 00:00:00.000,JERSON ANDRES ROJAS RODRIGUEZ,NaN,4 - SAN CRISTOBAL,32 - SAN BLAS,BELLAVISTA SUR,2,Natural,En nombre propio,NaN,NaN,GOOD AFTERNOON IS TO REQUEST THE REPAIR OF THE...,complaint,1
1,1,2992017,HABITAT,ACUEDUCTO - EAB,NaN,NaN,ACUEDUCTO - EAB,DIRECCION DE SERVICIO ADMINISTRATIVOS,SERVICIOS PUBLICOS,VISITAS TECNICAS,12 - BARRIOS UNIDOS,21 - LOS ANDES,ENTRERIOS,4.0,NaN,WEB,RECLAMO,INSTALACION DE MEDIDOR EL 8 DE NOVIEMBRE DE 20...,2017-01-02 00:00:00.000,SOLUCIONADO - POR RESPUESTA DEFINITIVA,2017-01-03 00:00:00.000,2017-01-24 14:26:01.929,15,2017-01-03 00:00:00.000,AMANDA OLIVA BOTERO SANCHEZ,ADULTO MAYOR,12 - BARRIOS UNIDOS,21 - LOS ANDES,ENTRERIOS,4,Natural,En nombre propio,NaN,NaN,"INSTALLATION OF METER ON NOVEMBER 8, 2016, THE...",complaint,1
2,2,3622017,HABITAT,ACUEDUCTO - EAB,NaN,NaN,ACUEDUCTO - EAB,DIRECCION DE SERVICIO ADMINISTRATIVOS,SERVICIOS PUBLICOS,DISPONIBILIDAD DE SERVICIO,2 - CHAPINERO,99 - CHAPINERO,CHAPINERO NORTE,5.0,NaN,WEB,DERECHO DE PETICIÓN DE INTERÉS PARTICULAR,EMPRESA DE ACUEDUCTO Y ALCANTARILLADO DE BOGOT...,2017-01-02 00:00:00.000,SOLUCIONADO - POR RESPUESTA DEFINITIVA,2017-01-03 00:00:00.000,2017-02-02 11:20:55.660,22,2017-01-03 00:00:00.000,JAIME MONCAYO ARENAS,ADULTO MAYOR,2 - CHAPINERO,99 - CHAPINERO,CHAPINERO NORTE,5,Natural,En nombre propio,NaN,NaN,"COMPANY OF AQUEDUCT AND SEWER OF BOGOTA, ESP\n...",complaint,1
3,3,3772017,HABITAT,ACUEDUCTO - EAB,NaN,NaN,ACUEDUCTO - EAB,DIRECCION DE SERVICIO ADMINISTRATIVOS,AMBIENTE,"MANTENIMIENTO DE SUMIDEROS, REJILLAS DE ALCANT...",2 - CHAPINERO,97 - CHICO LAGO,ANTIGUO COUNTRY,5.0,NaN,WEB,RECLAMO,FAVOR REPONER LA REJILLA DE SUMNIDERO QUE SE E...,2017-01-02 00:00:00.000,SOLUCIONADO - POR RESPUESTA DEFINITIVA,2017-01-03 00:00:00.000,2017-02-02 11:55:16.434,22,2017-01-03 00:00:00.000,SERGIO VALDES BELTRAN,NaN,NaN,NaN,NaN,5,Natural,En nombre propio,NaN,NaN,PLEASE REPLACE THE GRILLE OF ABSNIDERO THAT IS...,complaint,1
4,4,3862017,HABITAT,ACUEDUCTO - EAB,NaN,NaN,ACUEDUCTO - EAB,DIRECCION DE SERVICIO ADMINISTRATIVOS,SERVICIOS PUBLICOS,RECUPERACIÓN DE ESPACIO PUBLICO POR OBRAS DE L...,1 - USAQUEN,14 - USAQUEN,USAQUEN,6.0,NaN,WEB,QUEJA,ME REFIERO A SU COMUNICACION S-2016-245954 LA ...,2017-01-02 00:00:00.000,SOLUCIONADO - POR RESPUESTA DEFINITIVA,2017-01-03 00:00:00.000,2017-01-16 10:47:02.935,9,2017-01-03 00:00:00.000,SERGIO VALDES BELTRAN,NaN,NaN,NaN,NaN,5,Natural,En nombre propio,NaN,NaN,I REFER TO YOUR COMMUNICATION S-2016-245954 WH...,complaint,1


In [6]:
df_w_labels.groupby(['Tema', 'complaint_binary'], as_index=False)['Sector'].count()

,Tema,complaint_binary,Sector
0,1 TRASLADO POR NO COMPETENCIA,0,16
1,1 TRASLADO POR NO COMPETENCIA,1,103
2,AMBIENTE,0,13
3,AMBIENTE,1,65
4,ARTE - CULTURA - PATRIMONIO,0,47
5,CIENCIA - TECNOLOGIA - COMUNICACIONES TIC,0,77
6,CIENCIA - TECNOLOGIA - COMUNICACIONES TIC,1,4
7,SERVICIOS PUBLICOS,0,52
8,SERVICIOS PUBLICOS,1,608


In [7]:
df_w_labels.groupby(['complaint_binary'], as_index=False)['Sector'].count()

,complaint_binary,Sector
0,0,207
1,1,788


In [8]:
def stem_tokens(tokens, stemmer):  
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

In [9]:
def tokenize(text):  
    # remove punctuation
    text = ''.join([c for c in text if c not in non_words])
    # tokenize
    tokens =  word_tokenize(text)

    # stem
    try:
        stems = stem_tokens(tokens, stemmer)
    except Exception as e:
        print(e)
        print(text)
        stems = ['']
    return stems

In [10]:
vectorizer = CountVectorizer(  
                analyzer = 'word',
                tokenizer = tokenize,
                lowercase = True,
                stop_words = spanish_stopwords)

In [11]:
X = df_w_labels['Asunto']
y = df_w_labels['complaint_binary']

In [12]:
# create training and testing vars
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [13]:
train_features = vectorizer.fit_transform([r for r in X_train])
test_features = vectorizer.transform([r for r in X_test])

/Users/geoffperrin/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  app.launch_new_instance()


In [14]:
train_features

<696x4714 sparse matrix of type '<type 'numpy.int64'>'
	with 28200 stored elements in Compressed Sparse Row format>

In [15]:
# Fit a naive bayes model to the training data.
# This will train the model using the word counts we computer, and the existing classifications in the training set.
nb = MultinomialNB()
nb.fit(train_features, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [16]:
# Now we can use the model to predict classifications for our test features.
predictions = nb.predict(test_features)

In [17]:
actual = y_test

# Generate the roc curve using scikits-learn.
fpr, tpr, thresholds = metrics.roc_curve(actual, predictions, pos_label=1)

# Measure the area under the curve.  The closer to 1, the "better" the predictions.
print("AUC of the predictions: {0}".format(metrics.auc(fpr, tpr)))

AUC of the predictions: 0.826495726496


## Now read in full complaint data

In [18]:
xl = pd.ExcelFile(data + "SDQS/SDQS 2017-enejun.xlsx")

In [19]:
df = xl.parse('DB Recibidos')

### Apply model built above to full set of complaints

In [20]:
full_features = vectorizer.transform([r for r in df['Asunto']])

/Users/geoffperrin/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  app.launch_new_instance()


In [21]:
complaint_pred = nb.predict(full_features)

In [22]:
complaint_pred

array([1, 1, 1, ..., 1, 1, 1])

In [23]:
df['complaint_pred'] = complaint_pred

In [24]:
complaints_by_tema = df.groupby(['Tema', 'complaint_pred'], as_index=False)['Sector'].count()

In [25]:
pivot = complaints_by_tema.pivot(index='Tema',columns='complaint_pred',values='Sector')

In [26]:
df_complaints_by_tema = pivot.reset_index()
df_complaints_by_tema.columns = ['Tema', 'non_complaint_count', 'complaint_count']

In [27]:
df_complaints_by_tema['pct_complaint'] = df_complaints_by_tema.complaint_count / \
(df_complaints_by_tema.non_complaint_count + df_complaints_by_tema.complaint_count)

In [28]:
#replace nans with zeros
df_complaints_by_tema.fillna(0, inplace=True)

In [29]:
df_complaints_by_tema.sort_values('pct_complaint', ascending=False)

,Tema,non_complaint_count,complaint_count,pct_complaint
29,FINANCIERO,15.0,1442.0,0.989705
52,Obras en Espacio público,1.0,64.0,0.984615
64,SOLICITUD DE VALORACION FORESTAL,2.0,103.0,0.980952
63,SERVICIOS PUBLICOS,120.0,3644.0,0.968119
37,GESTION DEL RIESGO,6.0,131.0,0.956204
44,"IMPUESTOS, TASAS Y CONTRIBUCIONES",59.0,1011.0,0.944860
61,SEGURIDAD LOCAL Y PROBLEMÁTICAS ASOCIADAS,7.0,102.0,0.935780
65,SUBSIDIOS,250.0,3393.0,0.931375
27,ESPACIO PUBLICO,110.0,1485.0,0.931034
16,CARCEL DISTRITAL,115.0,1536.0,0.930345


In [30]:
114003.0 / (114003+31977)

0.7809494451294698